In this notebook, we explore how GPT2's responses are handles by our recovered reward function, and compare to the actor. 

We use the resource for accessing GPT2 https://demo.allennlp.org/next-token-lm


In [ ]:
# TODO: Import model

#Import get raw action

# Import get state TODO

In [ ]:
"""
Main.py file for GAIL implementation on dialog datasets.

Uses command line arguments to maximize flexibility, and run many options in parallel
"""

import sys 
sys.path.append('../src')
import os
import pickle
import argparse
import numpy as np
from collections import deque

import torch
import torch.optim as optim
from tensorboardX import SummaryWriter 

from models.actor import Actor
from models.critic import Critic
from models.discriminator import Discriminator
from GAIL import *

from dialog_environment import DialogEnvironment

device='cuda' # for now


parser = argparse.ArgumentParser(description='Limitation Learning')

parser.add_argument('--load_model', 
                    type=str, default=None, 
                    help='path to load the saved model')

parser.add_argument('--gamma', 
                    type=float, default=0.99, 
                    help='discounted factor (default: 0.99)')

parser.add_argument('--lamda', 
                    type=float, default=0.98, 
                    help='GAE hyper-parameter (default: 0.98)')


parser.add_argument('--learning_rate', 
                    type=float, default=3e-4, 
                help='learning rate of models (default: 3e-4)')

parser.add_argument('--l2_rate', 
                type=float, default=1e-3, 
                help='l2 regularizer coefficient (default: 1e-3)')

parser.add_argument('--clip_param', 
                type=float, default=0.2, 
                help='clipping parameter for PPO (default: 0.2)')

parser.add_argument('--discrim_update_num', 
                type=int, default=2, 
                help='update number of discriminator (default: 2)')

parser.add_argument('--actor_critic_update_num', 
                type=int, default=10, 
                help='update number of actor-critic (default: 10)')

parser.add_argument('--total_sample_size', 
                type=int, default=2048, 
                help='total sample size to collect before PPO update (default: 2048)')

parser.add_argument('--batch_size', 
                type=int, default=128, 
                help='batch size to update (default: 128)')

parser.add_argument('--suspend_accu_exp', 
                type=float, default=None,
                help='accuracy for suspending discriminator about expert data (default: None)')

parser.add_argument('--suspend_accu_gen', 
                type=float, default=None,
                help='accuracy for suspending discriminator about generated data (default: None)')

parser.add_argument('--max_iter_num', 
                type=int, default=4096,
                help='maximal number of main iterations (default: 4000)')

parser.add_argument('--seed', 
                type=int, default=42,
                help='random seed (default: 500)')

parser.add_argument('--logdir', 
                type=str, default='logs/EXPERIMENTNAME',
                help='tensorboardx logs directory (default: logs/EXPERIMENTNAME)')

parser.add_argument('--hidden_size', 
                type=int, default=128,
                help='New sequence length of the representation produced by the encoder/decoder RNNs. (default: 1024)')
parser.add_argument('--num_layers', 
                type=int, default=2,
                help='Number of layers in the respective RNNs (default: 2)')

parser.add_argument('--seq_len', 
                type=int, default=5,
                help='length of input and response sequences (default: 60, which is also max)')
parser.add_argument('--input_size', 
                type=int, default=300,
                help='DO NOT CHANGE UNLESS NEW EMBEDDINGS ARE MADE. Dimensionality of embeddings (default: 300)')

#args = parser.parse_args()


from argparse import Namespace
args = Namespace(load_model=None,
                render=False,
                gamma=.99,
                lamda=.98,
                learning_rate=.001,
                l2_rate=.001,
                clip_param=.2,
                discrim_update_num=2,
                actor_critic_update_num=10,
                total_sample_size=1024,
                batch_size=512,
                suspend_accu_exp=None,# won't stop
                suspend_accu_gen=None,
                max_iter_num=4000,
                seed=500,
                logdir='logs/newestOVERFIT',
                 hidden_size=128,
                 num_layers=3,
                 seq_len=5,
                 input_size=50
                )


env = DialogEnvironment()

torch.manual_seed(args.seed)

#TODO
actor = Actor(hidden_size=args.hidden_size,num_layers=args.num_layers,device='cuda',input_size=args.input_size,output_size=args.input_size)
critic = Critic(hidden_size=args.hidden_size,num_layers=args.num_layers,input_size=args.input_size,seq_len=args.seq_len)
discrim = Discriminator(hidden_size=args.hidden_size,num_layers=args.hidden_size,input_size=args.input_size,seq_len=args.seq_len)

actor.to(device), critic.to(device), discrim.to(device)

actor_optim = optim.Adam(actor.parameters(), lr=args.learning_rate)
critic_optim = optim.Adam(critic.parameters(), lr=args.learning_rate, 
                          weight_decay=args.l2_rate) 
discrim_optim = optim.Adam(discrim.parameters(), lr=args.learning_rate)

# load demonstrations

writer = SummaryWriter(args.logdir) # saved_model/

if args.load_model is not None: #TODO
    saved_ckpt_path = #os.path.join(os.getcwd(), 'save_model', str(args.load_model))
    ckpt = torch.load(saved_ckpt_path)

    actor.load_state_dict(ckpt['actor'])
    critic.load_state_dict(ckpt['critic'])
    discrim.load_state_dict(ckpt['discrim'])

In [1]:
import os

In [3]:
os.listdir('../src/save_model/')[1]

'sundayv2_ckpt_0.pth.tar'

In [10]:
saved_ckpt_path = '../src/save_model/' + os.listdir('../src/save_model/')[1]

In [7]:
import torch

In [12]:
ckpt = torch.load(saved_ckpt_path) # try this soon... 

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [11]:
saved_ckpt_path

'../src/save_model/sundayv2_ckpt_0.pth.tar'